# Purpose: gather statistics on question lengths, passage lengths, etc

In [2]:
from collections import Counter
import string
import re
import argparse
import json
import sys
import numpy as np
import nltk
import random
import math
import os
import pickle
from tqdm import tqdm, trange

import pdb

from pytorch_pretrained_bert.tokenization import (BasicTokenizer,
                                                  BertTokenizer,
                                                  whitespace_tokenize)

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

100%|██████████| 231508/231508 [00:03<00:00, 67234.41B/s]


In [4]:
def pickler(path,pkl_name,obj):
    with open(os.path.join(path, pkl_name), 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def unpickler(path,pkl_name):
    with open(os.path.join(path, pkl_name) ,'rb') as f:
        obj = pickle.load(f)
    return obj

In [7]:
TRAINING = False

out_pkl_path = "./"

if(TRAINING):
    file_path = "../../../hotpotqa/hotpot_train_v1.json"
#     out_pkl_name = "preprocessed_train.pkl"
else:
    file_path = "../../../hotpotqa/hotpot_dev_distractor_v1.json"
#     out_pkl_name = "preprocessed_dev.pkl"

In [8]:
with open(file_path, encoding='utf8') as file:
    dataset = json.load(file)

In [9]:
def tokenize(text):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))

In [10]:
questions = []
paragraphs = [] 
supporting_facts = []


for item_index, item in enumerate(tqdm(dataset)):
#     if(item_index in problem_indices):
#         continue
    question = tokenize(item["question"])
    questions.append(question)
    paragraph_names = []
    paragraph_text = []
    for i,para in enumerate(item["context"]):
        para_name = para[0]
        para_sents = para[1]
        paragraph_names.append(para_name)
        paragraph_text.append([tokenize(s) for s in para_sents])
    paragraphs.append(paragraph_text)
    supp_fact_list = []
    for sup_fact in item["supporting_facts"]:
        para_name = sup_fact[0]
        supporting_fact_index = sup_fact[1] 
        para_index = paragraph_names.index(para_name)
        supp_fact_list.append([para_index, supporting_fact_index])
    supporting_facts.append(supp_fact_list)

100%|██████████| 7405/7405 [02:10<00:00, 56.70it/s]


In [11]:
question_lengths = np.array([len(q) for q in questions])

print("Avg question len:{}".format(question_lengths.mean()))
print("min question len:{}".format(question_lengths.min()))
print("max question len:{}".format(question_lengths.max()))

Avg question len:19.590141796083728
min question len:7
max question len:65


In [20]:
max_question_len = 30
np.sum(np.greater(question_lengths,max_question_len))/question_lengths.shape[0]

0.06927751519243754

In [21]:
document_lengths = []

for doc in paragraphs:
    doc_len = 0
    for para in doc:
        for sent in para:
            doc_len += len(sent)
    document_lengths.append(doc_len)

document_lengths = np.array(document_lengths)


print("Avg document len:{}".format(document_lengths.mean()))
print("min document len:{}".format(document_lengths.min()))
print("max document len:{}".format(document_lengths.max()))

Avg document len:1194.0945307224847
min document len:66
max document len:3222


In [43]:
max_document_len = 1700
np.sum(np.greater(document_lengths,max_document_len))/document_lengths.shape[0]

0.06144496961512492

In [33]:
def find_num_chunks(max_sequence_length, max_question_length, max_document_length):
    s1 = max_sequence_length - max_question_length
    num_chunks = max_document_length / s1
    return num_chunks

In [44]:
find_num_chunks(max_sequence_length=509, max_question_length=30, max_document_length=1700)

3.549060542797495